In [1]:
from dao.mongodbdao import MongoDBDao
from tfidf.TF_IDF import TF_IDF
import numpy as np 

dao = MongoDBDao({
    "host": "127.0.0.1",
    "db": "default",
    "wordindex_collection": "wordindex",
    "pagedetails_collection": "pagedetails",
    "username": "root",
    "password": "spezanw"
})

# Mindestähnlichkeit festlegen?

In [2]:
tfidf = TF_IDF(dao)

TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.276386.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.461948.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.587702.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.85381.php'} contains no words


In [3]:
urls = [p["_id"] for p in dao.pagedetails_collection.find({})]

In [4]:
urls_with_similar_articles = []
similar_articles = []
for i, url in enumerate(urls):
    articles_with_sims = tfidf.getSimilarArticles(url, n_results=20, return_sims=True, min_sim=None)
    urls_with_similar_articles.append((url, articles_with_sims))
    similar_articles.append(articles_with_sims)
    if (i+1)%100 is 0:
        print("calculating ...", ((i+1)/len(urls))*100, "%")

calculating ... 1.001401962747847 %
calculating ... 2.002803925495694 %
calculating ... 3.0042058882435407 %
calculating ... 4.005607850991388 %
calculating ... 5.007009813739235 %
calculating ... 6.0084117764870815 %
calculating ... 7.009813739234929 %
calculating ... 8.011215701982776 %
calculating ... 9.012617664730623 %
calculating ... 10.01401962747847 %
calculating ... 11.015421590226318 %
calculating ... 12.016823552974163 %
calculating ... 13.01822551572201 %
calculating ... 14.019627478469857 %
calculating ... 15.021029441217703 %
calculating ... 16.02243140396555 %
calculating ... 17.023833366713397 %
calculating ... 18.025235329461246 %
calculating ... 19.02663729220909 %
calculating ... 20.02803925495694 %
calculating ... 21.029441217704786 %
calculating ... 22.030843180452635 %
calculating ... 23.03224514320048 %
calculating ... 24.033647105948326 %
calculating ... 25.035049068696175 %
calculating ... 26.03645103144402 %
calculating ... 27.03785299419187 %
calculating ... 

In [13]:
t1 = []
for i, a in enumerate(similar_articles):
    try:
        t1.append(a[0])
    except IndexError:
        print("Error on index", i)
t1_sims = [a[1] for a in t1]

Error on index 3735
Error on index 5198
Error on index 7164
Error on index 9781


In [23]:
print("minimale Ähnlichkeit:",np.min(t1_sims))
print("maximale Ähnlichkeit:",np.max(t1_sims))

minimale Ähnlichkeit: 0.06401646358730045
maximale Ähnlichkeit: 0.9770869715556777


In [125]:
urls[1110]

'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.386214.php'

In [129]:
tfidf.term_idfs["mit"]
tfidf.term_idfs["raub"]

1.3639513030614177

In [84]:
def get_term_idfs_for_url(url, top):
    pd = dao.getPageDetails(url)
    words = pd["lemma_words"]
    
    term_idfs = {}
    for word in words:
        term_idfs[word] = tfidf.term_idfs[word]
    
    s = sorted(term_idfs.items(), key=lambda x: x[1])[::-1]
    if len(s) < top:
        return s
    else:
        return s[:top]

In [101]:
import math
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def get_articles_on_simlarity_intervals(results):
    values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    #idfs = [e[1] for e in terms_idf]
    sims = [e[1] for e in results]
    nearest = []
    last_sim = -1
    for val in values:
        current_nearest = find_nearest(sims, val)
        i = np.argwhere(sims==current_nearest)[0][0]
        if not math.isclose(current_nearest, last_sim, rel_tol=1e-09):
            nearest.append(results[i])
            last_sim=current_nearest
    return nearest

In [102]:
import pprint

def get_result_info(url):
    pprint.pprint(get_term_idfs_for_url(url, top=15))
    results = tfidf.getSimilarArticles(url, n_results=600, return_sims=True, min_sim=0.001)
#print(results)
    pprint.pprint(get_articles_on_simlarity_intervals(results))

In [127]:
test_urls = ["https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.408478.php", #0.4
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.502250.php", #0.168
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.731596.php", #0.195
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.744108.php", #>0.14!!!
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.344120.php", #0.197
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.344571.php", #0.2
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.352960.php", #0.197
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.381353.php", #0.09
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.378166.php", #0.1
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.352953.php", #>0.17!!!
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.535381.php", #>0.14!!!
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.535584.php", #0.197
             "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.386214.php", #0.2155
            ]
for t in test_urls:
    get_result_info(t)
    print("\n____________________________________________________________________________\n")
    
mins = [0.4, 0.168, 0.195, 0.197, 0.2, 0.197, 0.09, 0.1, 0.197, 0.2155]
np.mean(mins)

[('videothek', 2.720681448923501),
 ('örtlich', 2.3462225361009863),
 ('bismarckstraße', 2.3273371919406123),
 ('erlangen', 2.021711444587482),
 ('ziel', 1.97005127219112),
 ('anwesend', 1.94635660639291),
 ('mitternacht', 1.5696827698739217),
 ('maskiert', 1.5538308466027322),
 ('zuständig', 1.515135210529544),
 ('herausgabe', 1.4335872312028122),
 ('schusswaffe', 1.3549964604084912),
 ('überfall', 1.2884719308810542),
 ('charlottenburg', 1.2630385475996873),
 ('raubkommissariat', 1.161215827968704),
 ('geschäft', 1.1475654491465634)]
[('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.569055.php',
  0.11388353415484288),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.649789.php',
  0.1998023321552386),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.290462.php',
  0.3010079901655242),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.537034.php',
  0.40030573381896684),
 ('https://www.berlin.de/polizei/polizeimeld

0.19595